# Redes RBF - _Radial Basis Function_
---

1. [Teorema de Cover](#Cover)<br>
    1.1 [Separabilidade](#Separabilidade) <br>
    1.2 [Entendendo o Teorema](#entendendo)
2. [Interpolação](#Interpolacao) <br>
    2.1 [Teorema de Micchelli](#micchelli)
3. [Camada de Entrada](#entrada) <br>
    3.1 [Pré-Processamento](#pre) <br>
    3.2 [Normalização](#normalização) 
4. [Parametrização](#parametrizar)
5. [Camada de Saida](#saida)
6. [Análise](#análise) <br>
    6.1 [K-Means](#k-means) <br>
    6.2 [Quantidade de RBF's x Acertos da Rede](#acertos) <br>
    6.2 [RBF x MLP](#disputa) <br>
7. [Bibliotecas Utilizadas e Bibliografia](#biblio)

In [3]:
import numpy as np
import numpy.linalg as LA

from sklearn.metrics import accuracy_score      # métrica de avaliação de resultados
from sklearn.cluster import KMeans              # algoritmo de clusterização
from sklearn.preprocessing import MinMaxScaler  # método para ajuste dos dados
from sklearn.model_selection import KFold       # Cross validator

from scipy.spatial.distance import euclidean    # Método para calcular a dist euclidiana entre dois pontos     
from scipy.spatial import distance_matrix       # Método para calcular a matriz de distância entre dois arrays

## 1. Teorema de Cover <a name="Cover"></a>
---

Uma **rede de função de base radial (RBF)** é usada para realizar uma tarefa _complexa_ de classificação de padrões.
A ideia é, basicamente, transformar os dados para um espaço de maior dimensionalidade, de uma forma não-linear.

Para justificarmos o motivo é necessário entender o _Teorema de Cover_:

> Um problema complexo de classificação de padrões dispostos não linearmente em um espaço de alta dimensão tem maior probabilidade de ser linearmente separável do que em um espaço de baixa dimensionalidade.
    
Primeiramente vamos entender o conceito de separabilidade.    
    
### 1.1 Separabilidade <a name="Separabilidade"></a>   

Seja $\mathscr{H} = \{x_{1},...,x_{N}\} $ um conjunto de $N$ entradas (_amostras_) divididias em duas classes $\mathscr{H}_1$ e $\mathscr{H}_2$, portanto, problema de classificação binário e considere uma família de superfícies onde cada uma divide naturalmente um conjunto de entradas em duas regiões. Portanto, $\mathscr{H}$ é separável se existe _uma superfície_ da família que separe os pontos de $\mathscr{H}_1$ dos pontos de $\mathscr{H}_2$ em duas regiões. Como mostra a imagem a seguir:

![dataset_1](./imgs/dataset_1.png)

Em seguida, $\forall{x} \in \mathscr{H}$ definimos um vetor $\varphi(x)$ constituído de funções reais $\{\varphi(x)_i|i=1,...,m_1\}$ como mostrado a seguir:

$$ \varphi(x) = [\varphi_1(x), \varphi_2(x), ..., \varphi_{m_1}(x)]^{T} $$

> $x \in \mathbb{R}^{m_0}$ : o vetor de entrada $x$ pertence a um espaço de dimensão $m_0$.

O vetor $\varphi(x)$ mapeia o vetor de entrada $x$ num espaço de dimensionalidade $m_1$, onde normalmente $\mathbb{R}^{m_0} < \mathbb{R}^{m_1}$, ou seja, mapeia para um espaço com maior dimensionalidade. Na figura a seguir, podemos ver pontos em $\mathbb{R}^2$ sendo mapeados em $\mathbb{R}^3$:

![dimension_1](./imgs/dimension_1.jpg)

> $\boldsymbol{\varphi_i(x)}$ : é uma _função oculta_ e o espaço que abrange estas funções ocultas $\{\varphi_i(x)\}^{m_1}_{i=1}$ é o o _espaço oculto_ ou _espaço de características_.

Um conjunto de dados com duas classes $\mathscr{H}_1$ e $\mathscr{H}_2$ é dito _separável_ por $\varphi$ se existir um vetor $\boldsymbol{w}$ de dimensão $m_1$ para o qual podemos escrever:

$$
\begin{split}
    w^T \cdot \varphi(x) > 0,\quad x \in \mathscr{H}_1 \\
    w^T \cdot \varphi(x) < 0,\quad x \in \mathscr{H}_2
\end{split}
$$

O hiperplano separador, que descreve a superfície de separação no espaço oculto $\varphi$, é definido por:

$$ w^T \cdot \varphi(x) = 0 $$

### 1.2 Entendendo o teorema <a name="entendendo"></a>

Em um experimento probabilístico, a separabilidade de um conjunto de padrões se torna um **evento aleatório** que depende da dicotomia escolhida e da distribuição espacial dos dados.

Suponha que:

- Os padrões $\boldsymbol{x_1, x_2, ..., x_N}$ sejam escolhidos independentemente, de acordo com uma medida de probabilidade imposta no espaço de entrada.
- Todas as dicotomias de $\mathscr{H} = \{x_i\}^{N}_{i=1}$ são equiprováveis.

A equação abaixo representa a probabilidade de que uma dicotomia particular escolhida ao acaso seja separável por $\varphi$, onde a classe de superfícies de separação escolhida tenha $m_1$ graus de liberdade, i.e., está em $\mathbb{R}^{m_1}$.

$$ P(N,m_1) = \left(\frac{1}{2}\right)^{N-1} \cdot \sum_{m=0}^{m_1 - 1}
    \left(\!
        \begin{array}{c}
            N-1 \\
            m
        \end{array}
    \!\right) $$

Esta equação personifica a essência do **Teorema da separabilidade de Cover**.

## 2. Interpolação <a name="Interpolacao"></a>

Basicamente, um mapeamento não-linear é usado para transformar um problema de classificação não-linearmente separável em um problema linearmente separável. De maneira análoga, podemos trasformar um problema de filtragem não-linear difícil em um problema mais fácil que envolve filtragem linear.

A rede é projetada para realizar um **mapemento não-linear** do espaço de entrada para o espaço oculto, seguido de um **mapemanto linear** do espaço oculto para o espaço de saída.

Se $m_0$ representa a dimensão do espaço de entrada, então, de uma maneira global, a rede representa um mapeamento de um espaço de dimensionalidade $m_0$ para um espaço unidimensional, da seguinte forma:

$$ s: \mathbb{R}^{m_o}→\mathbb{R}^1 $$

Onde $s$ é uma _hipersuperfície_ $\Gamma \subset \mathbb{R}^{m_0+1}$. Na prática, esta superfície é desconhecida  e os dados de treinamento estão normalmente contaminados com ruídos.

As fases de treinamento e generalização podem ser vistas, respectivamente, da seguinte forma:

1. __Treinamento__: constitui uma otimização de um procedimento de ajuste para a superfície $\Gamma$, baseada nos pontos dos dados conhecidos apresentados à rede na forma de amostras de entrada-saída.
2. __Generalização__ : sinônimo de interpolação entre os pontos de dados, com a interpolação sendo formada ao longo da superfície restrita gerada pelo procedimento de ajuste, com a aproximação sendo ótima à superfície verdadeira $\Gamma$.

Desta forma, somos levados à teoria da _interpolação multivariada_ em um espaço de alta dimensionalidade:

> Dado um conjunto de $N$ pontos diferentes $\{x_i \in \mathbb{R}^{m_0} | i=1,2,...,N\}$ e um conjunto correspondente de $N$ numeros reais $\{d_i \in \mathbb{R}^{1} | i=1,2,...,N\}$, encontre uma função $F:\mathbb{R}^{m_0}→\mathbb{R}^{1}$ que satisfaça a condição de interpolação:
$$F(x_i) = d_i, \quad i=1,2,...,N$$

A superfície de interpolação (i.e., a função $F(x_i$) é obrigada a passar por todos os pontos do _dataset_. 

A técnica aqui aprensetada (**RBF**) consiste em encontrar a melhor função $F$ que tem a seguinte forma:
$$ F(\boldsymbol{x}) = \sum_{i=1}^{N}w_i \cdot \varphi(\parallel x - x_i \parallel) $$

Onde, $\{\varphi(\parallel x - x_i\parallel)| i=1,...,N\}$ é um conjunto de $N$ funções (geralmente não-lineares) arbitrárias, conhecidas como **funções de base radial**, e $\parallel . \parallel$ representa uma **norma** que normalmente é euclidiana. Os pontos de dados conhecidos $x_i \in \mathbb{R}^{m_0}|i=1,...,N$ são tomados como **centróides** das funções de base radial.

Inserindo a _condição de interpolação_ à útlima equação, temos que:

$$
\begin{bmatrix}
    \varphi_{11} & \varphi_{12} & ... & \varphi_{1N} \\[0.3em]
    \varphi_{21} & \varphi_{22} & ... & \varphi_{2N} \\[0.3em]
    \vdots & \vdots & \ddots & \vdots \\[0.3em]
    \varphi_{N1} & \varphi_{N2} & ... & \varphi_{NN} \\[0.3em]
\end{bmatrix}
%
\begin{bmatrix}
    w_1 \\[0.3em]
    w_2 \\[0.3em]
    \vdots \\[0.3em]
    w_N \\[0.3em]
\end{bmatrix}
%
=
\begin{bmatrix}
    d_1 \\[0.3em]
    d_2 \\[0.3em]
    \vdots \\[0.3em]
    d_N \\[0.3em]
\end{bmatrix}
$$

onde

$$ \varphi_{ij} = \varphi(\parallel x_j - x_i\parallel), i=1,...,N $$

- $ \boldsymbol{d} = [d_1, ..., d_N]^T $: representa o _vetor de respostas desejadas_.
- $ \boldsymbol{w} = [w_1, ..., w_N]^T $: representa o _vetor de pesos lineares_.
- $N$: tamanho do conjunto de amostras.
- $ \Phi = \{\varphi{ji}|(j,i) = 1,...,N\}$: representa a _matriz de interpolação_.

Por fim,

$$
\begin{split}
   \Phi \cdot w = d \\
    w = \Phi^{-1} \cdot d
\end{split}
$$

> **Importante**: Precisamos ter certeza que a matriz de interpolação $\Phi$ é não-singular, i.e., $det(\Phi) \neq 0$.

### 2.1 Teorema de Micchelli <a name="micchelli"></a>

> Considere que $\{x_i\}^{N}_{i=1}$ seja um conjunto de pontos distintos em $\mathbb{R}^{m_o}$. Então, a matriz de interpolação $\Phi_{N \times N}$, cujo o $ji$-ésimo elemento é $\varphi_{ji} = \varphi(\parallel x_j - x_i \parallel)$, é não-singular.

Neste trabalho usaremos **Funções gaussianas**, que detém as características necessárias:

$$ \varphi(r) = exp(-\frac{r²}{2\sigma²})$$

## 3. Camada de Entrada <a name="entrada"></a>


O arquivo de entrada está sendo lido com o auxílio da biblioteca Numpy, que fornece estruturas de dados de alto desempenho, facilitando o uso de ferramentas para análise de dados, neste caso utilizamos da função map.


   A camada de entrada da RBF desenvolvida serve como um comunicador (lembrando que a camada de entrada não é um neurônio) entre os dados de entrada e a rede neural. Para uma base de dados de dimensão (1593 x 256), há duas possibilidades:
- Criar uma camada de entrada com dimensões (256 x 1). A cada iteração, a camada de entrada é responsável por percorrer a base de entrada. No projeto, foi implementada camada de entrada com essas dimensões;
- Criar uma camada de entrada com dimensões (16 x 1). Neste caso, há necessidade de se usar multiplexador para tratar os dados de entrada antes de enviar à camada escondida.


Para verificarmos a capacadidade de separação em dimensionalidades altas, sem perda de generalização do Teorema de Cover, projetamos o espaço original de $m_0' = 256$ dimensões para um espaço de $m_0 = 16$ dimensões. Neste novo espaço aplicaremos as funções de base radial em maiores dimensionalidades para separar os dados.

### 3.1 Pré-Processamento <a name="pre"></a>

Para o Pré-Processamento dos dados realizamos um procedimento, de redução do tamanho dos parâmetros de entrada, transformando o qa linha de binários no seu decimal correspondente, tomemos como exemplo uma imagem definida por uma matriz 5x5: 

Para a entrada: 

    [ 0 0 0 0 0 ] -> corresponde ao decimal da 1 posição    
    [ 0 0 0 0 1 ] -> corresponde ao decimal da 2 posição    
    [ 0 0 0 1 1 ] ->             .             
    [ 0 0 1 0 0 ] ->             .             
    [ 1 0 1 0 1 ] -> corresponde ao decimal da 5 posição    
    
Note que o número de digítos será igual ao número de linhas da matriz. 

Transformaremos cada uma das linhas da matriz em um número decimal, e cada um dos vetores de classe, em seu decimal correspondente onde assim não perdemos valor semântico, e atenuaremos o problema do Teorema de Cover, do acrescimo de dimensões.

Inicialmente Nosso problema tem 256 dimensões, após o pré-processamento teremos como resultado 16 digítos e dimensões, atingindo nosso objetivo de tornar o problema menos complexo de ser resolvido.
    
Tome a seguinte imagem $4 \times 4$ como exemplo:    
    
![dataset_1](./imgs/flower.png)    
    
A transformação realiza uma mudança do número binário representado pela $i$-ésima linhas da imagem para o decimal correspondente. Portanto, de modo genérico, dada uma imagem $N \times N$:

$$
\begin{bmatrix}
    b_{11} & b_{12} & ... & b_{1N} \\[0.3em]
    b_{21} & b_{22} & ... & b_{2N} \\[0.3em]
    \vdots & \vdots & \ddots & \vdots \\[0.3em]
    b_{N1} & b_{N2} & ... & b_{NN} \\[0.3em]
\end{bmatrix}
%
=
\begin{bmatrix}
    D_1 \\[0.3em]
    D_2 \\[0.3em]
    \vdots \\[0.3em]
    D_N \\[0.3em]
\end{bmatrix}
$$

$$
\begin{bmatrix}
    B_{1} \\[0.3em]
    B_{2} \\[0.3em]
    \vdots \\[0.3em]
    B_{N} \\[0.3em]
\end{bmatrix}
%
=
\begin{bmatrix}
    D_1 \\[0.3em]
    D_2 \\[0.3em]
    \vdots \\[0.3em]
    D_N \\[0.3em]
\end{bmatrix}
$$
 
- $B_i = [b_{i1}, ..., b_{iN}] \quad \forall i \in [1,N]$: representa o vetor binário com os elementos correspondentes, onde $b_{ij} = (0,1)$.
- $D_i \quad \forall i \in [1,N]$: representa o decimal correspondente ao binário.
   
 

In [7]:
def preProcessing_Data(path):
    fp = open(path, 'r')

    X = [] # features
    Y = [] # labels

    for i in fp:
        x = []

        for j in range(16):
            aux = list(map(float, i.split()[16*j:16*(j+1)]))
            aux = np.array(aux, dtype='int')
            
            shift = np.arange(len(aux)-1, -1, -1)

            x.append(np.sum(aux << shift))             

        X.append(x)
        
        #y = np.array(list(map(float, i.split()[-10:])))
        #Y.append(np.argwhere(y == 1)[0][0])
        Y.append(list(map(float, i.split()[-10:])))

    fp.close()

    return (np.array(X, dtype=float),np.array(Y, dtype=int))

In [8]:
(X, Y) = preProcessing_Data("./data/semeion.data")

### 3.2 Normalização <a name="normalização"></a>

A normalização de dados de entrada é o processo pelo qual todos os dados são normalizados, ou seja, reduzido aos limites [0,1] ou [min,max]. Caso a normalização não seja realizada, os dados de entrada terão um efeito adicional sobre o neurônio, levando a decisões errada.

A fórmula de normalização, é:
    
$$ x_{new} = \left(\frac{x-x_{min}}{x_{max} - x_{min}}\right) \cdot (d_2-d_1) +d_1 $$

sendo:
- $x$ - o valor a ser normalizado;
- $[x_{max},x_{min}]$ - variação dos valores de x;
- $[d_1,d_2]$ - limite de redução de x.

In [9]:
scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X)

## 4. Parametrização <a name="parametrizar"></a>

A representação dos dados da rede está definida da seguinte maneira:

- Número de neurônios na camada escondida:

>Seguindo o teorema de Cover, para conseguirmos aumentar o número de dimensões precisamos que a quantidade de neurônios RBF na camada escodida, sejam de tamanho maior ou igual ao da entrada

> Tomemos como exemplo uma entrada definida em $\mathbb{R}^3$, para possibilitar que o algoritmo aumente o número de dimensões, precisamos ter pelo menos 4 neurônios na camada escondida.

- Vale notar que:

>  Diferente da MLP a rede RBF necessita de neurônios para o cálculo das funções gaussianas, separando o espaço de entrada, este fator em conjunto ao fator de cada neurônio produzir uma gaussiana para separação do espaço, podemos gerar overfitting caso tenhamos um neurônio para cada amostra, então a quantidade de neurônios necessária é menor.

- Posição dos centros de cada neurônio escondido:
> A posição dos centros será escolhida sobre a posição dos números, o algoritmo responsável será o k-means, comentaremos mais sobre na seção 7.1 [K-Means](#k-means)

- Parâmetros da função RBF (e.g. spread ou variância da gaussiana):
> def_RBF(X, Y, centers, size_rbf, size_output)

        - X : conjunto de dados;
        - Y : classes; 
        - centers : posição inicial dos centroídes;
        - size_rbf : quantidade de neuônios na camada escondida;
        - size_output : quantidade de neuônios na camada de saída.



- Pesos do(s) perceptron(s) da camada de saída.
>  Os pesos são iniciados aleatoriamente seguindo uma distribuição normal, devido a gaussiana, e são atualizados seguindo a regra delta.


## 5. Camada de saída <a name="saida"></a>

A camada de saída é composta por 10 neurônios seguindo os modelos do neurônio de Rosenblatt:

> Tal escolha foi motivada pela quantidade do número de saídas possíveis, já que o objetivo da rede neural é a identificação de dígitos de 0 a 9, decidimos por optar pela quantidade  de neurônios iguais a quantidade de dígitos a serem identificados. 
    
**- mostrar código fonte onde as equações estão sendo feitas**


## 6. Análise <a name="análise"></a>  

Utilizamos essa seção para realizar análises pertinentes ao trabalho

### 6.1 K-Means <a name="k-means"></a>  

Utilizamos o método k-means para decidir a posição dos centros das RBFs 

**- parte do código mostrando o k-means sendo usado**

O algoritmo k-means funciona da seguinte maneira:

> Etapa 1: Determinação dos centros de clusters iniciais <br>

A etapa 1 os centros iniciais são escolhidos aleatoriamente e ajustados pelo calculo da distância entre o centro e as demais pontos da imagem repetindo até encontrar o centro do conjunto K,

> Etapa 2: Iteração do conjunto de dados de treinamento e cálculo dos centros de clusters <br>

Onde no na etapa 2 é utilizado todo o conjunto de dados, ajustando os centros para a posição real (considerando todo o conjunto de dados), acreditamos que o fator aleatório inicial seguido dos ajustes calculados na segunda etapa do algoritmo são uma boa maneira de determinar os centros das RBFs.


### 6.2 Quantidade de RBF's x Acertos da Rede <a name="acertos"></a>  

Tabela acertos


### 6.3 RBF x MLP <a name="disputa"></a>  


Gráfico de desempenho 


## 7. Bibliotecas Utilizadas e Bibliografia <a name="biblio"></a>  

As bibliotecas necessárias para a execução do algoritmo são:
- [Numpy](https://www.numpy.org/): indexação e manipulação das matrizes e vetores. <br>
- [scikit-learn](https://scikit-learn.org/stable/): utilização de métodos de validação, pré-processamento e agrupamento. <br>
- [SciPy](https://www.scipy.org/): utilizado para cálculo de distâncias.

Bibliografia
- Material disponibilizado em sala de aula
   - [Slide - Cap 6: Redes de Função de Base Radial](https://ae4.tidia-ae.usp.br/access/content/group/3084400d-5d19-4ca6-84c5-8101fbe15260/Slides-Aulas/SCC0570Cap6_2019.pdf) <br>
- Redes Neurais - Princípios e Práticas, 2ª Edição, Simon Haykin. 
   
Trabalhos estudados 
- https://www.researchgate.net/figure/Figura-211-Exemplo-de-funcao-de-vizinhanca-gaussiana-bidimensional-com-neuronio_fig6_201549563
- https://docs.aws.amazon.com/pt_br/sagemaker/latest/dg/algo-kmeans-tech-notes.html
- https://matheusfacure.github.io/2017/02/20/MQO-Gradiente-Descendente/
- https://www.mql5.com/pt/articles/497
- https://en.wikipedia.org/wiki/Radial_basis_function_kernel
